In [26]:
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

data_path = "../data/featureEngineering/"
df_train = pd.read_csv(data_path + 'house_train.csv')
df_test = pd.read_csv(data_path + 'house_test.csv')

#將資料成為訓練/預設用模式
train_Y = np.log1p(df_train["SalePrice"])
ids = df_test["Id"]
df_train = df_train.drop(["Id","SalePrice"],axis = 1)
df_test  = df_test .drop(["Id"],axis = 1)
df = pd.concat([df_train,df_test])
print(df.shape)
df.head()

(2919, 79)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


* 查詢各欄位空缺值數量 

In [27]:
# 檢查欄位缺值數量 (去掉.head()可以顯示全部)
df.isnull().sum().sort_values(ascending=False).head()

PoolQC         2909
MiscFeature    2814
Alley          2721
Fence          2348
FireplaceQu    1420
dtype: int64

In [28]:
num_features = []
for dtype,feature in zip(df.dtypes,df.columns):
    if dtype == "float64" or dtype == "int64":
        num_features.append(feature)
print(f"{len(num_features)} Numeric Features : {num_features}\n")

36 Numeric Features : ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']



In [29]:
#只過濾出數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,...,548.0,0,61,0,0,0,0,0,2,2008
1,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,...,460.0,298,0,0,0,0,0,0,5,2007
2,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,...,608.0,0,42,0,0,0,0,0,9,2008
3,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,...,642.0,0,35,272,0,0,0,0,2,2006
4,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,...,836.0,192,84,0,0,0,0,0,12,2008


* 觀察替換不同補缺方式, 對於特徵的影響

### [交叉驗證(cross_val_score)]
#### 為了避免依賴某一特定的訓練和測試資料產生偏差。
1. k-folder cross-vailation
2. kk folder cross-vaildation
3. least-one-out cross-validation
4. 10-fold corss validation
5. 參考:
    - [K-Fold Cross Validation and GridSearchCV in Scikit-Learn](https://randomforests.wordpress.com/2014/02/02/basics-of-k-fold-cross-validation-and-gridsearchcv-in-scikit-learn/)
    - [機器學習中的交叉驗證](https://blog.csdn.net/lhx878619717/article/details/49079785)

* k-folder cross-vailation
將資料分成5等份，其中第1等分用來當作驗證的測試資料，其餘4份拿來訓練，下一輪繼續將第2等分拿來當作驗證的測試資料，其餘4份一樣拿來訓練，總共做5次。
藉著將5次的準確性(Accuracy)平均，而這個得到這個數值就是準確度較沒有偏差。
* 下面 cv=5 分成五組

In [30]:
# 空值補 -1, 做線性迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.8466400643386441

In [31]:
# 空值補 0
df_0  = df.fillna(0)
train_X = df_0[:train_num]
estimator = LinearRegression()
cross_val_score(estimator,train_X,train_Y,cv=5).mean()

0.8466118155868791

In [32]:
# 空值補平均
df_mn= df.fillna(df.mean())
train_X = df_mn[:train_num]
estimator = LinearRegression()
cross_val_score(estimator,train_X,train_Y,cv=5).mean()

0.8442642432201273

* 觀察替換不同特徵縮放方式, 對於特徵的影響

In [33]:
# 空值補 -1, 搭配最大最小化
df = df.fillna(-1)
df_temp = MinMaxScaler().fit_transform(df)
train_X = df_temp[:train_num]
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.8466363829279121

In [34]:
# 空值補 -1, 搭配標準化
df_temp = StandardScaler().fit_transform(df)
train_X = df_temp[:train_num]
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.8465369221570974

# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [82]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = "../data/featureEngineering/"
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [83]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [84]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [66]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6960299128976762

In [67]:
# 空值補 0
df_0 = df.fillna(0)
train_X = df_0[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6971535084032942

In [68]:
# 空值補 平均數
df_mn = df.fillna(df.mean())
train_X = df_mn[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6981761033723469

In [69]:
# 空值補 中位數(Median)
df_med = df.fillna(df.median())
train_X = df_med[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6992934218081011

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [86]:
# 空值補 -1, 搭配最大最小化
df_t = df.fillna(-1)
df_temp = MinMaxScaler().fit_transform(df_t)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
display(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.6971346062663598

In [87]:
# 空值補 -1, 搭配標準化
df_temp = StandardScaler().fit_transform(df_t)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
display(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.6982582017719778

In [81]:
score = pd.DataFrame(columns=["Fill","Scaler","Score"])
for fill_num, scaler in [(f,s) for f in [(-1,"-1"), (0, "0"), (df.mean(),"mean")] for s in [(None,"None"), (MinMaxScaler,"MinMaxScaler"), (StandardScaler,"StandardScaler")]]:
    df_temp = df.fillna(fill_num[0])
    train_X = df_temp[:train_num]
    if scaler[0]:
        s = scaler[0]()
        train_X = s.fit_transform(train_X)
    estimator = LogisticRegression()
    score = score.append(pd.DataFrame([[fill_num[1],scaler[1],cross_val_score(estimator, train_X, train_Y, cv=5).mean()]],columns=["Fill","Scaler","Score"]),ignore_index=True)
score.sort_values("Score",ascending=False)

,Fill,Scaler,Score
4,0,MinMaxScaler,0.707234
1,-1,MinMaxScaler,0.703864
2,-1,StandardScaler,0.698258
5,0,StandardScaler,0.698258
7,mean,MinMaxScaler,0.698227
6,mean,None,0.698176
3,0,None,0.697154
0,-1,None,0.696030
8,mean,StandardScaler,0.695941
